# Stopwords EU

In [1]:
# Data processing
import json
import numpy as np
import pandas as pd
import re

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import stopwords

#getting german stopwords:
german_stop_words = stopwords.words('german')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JVoigt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JVoigt\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JVoigt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Funcoes para limpar as palavras:

def remove_stopwords(element):
    if isinstance(element, str):
        if element in german_stop_words:
            return ''
        else:
            return element

def replace_char(x):
  return re.sub('[^a-zA-Z\süöäß]', '', x)

def clean_corpus(s):
    s = s.apply(lambda x: x.lower() if isinstance(x, str) else x)     #tolower:
    idx = s.apply(lambda x: isinstance(x, str))                       # Criar um índice booleano para filtrar os valores não-caracteres
    s = s.drop(idx[~idx].index)                                       # Remover os valores não-caracteres
    s = s.apply(replace_char)                                         # remover as partes de palavras que nao sao caracteres ou espacos
    idx = s.apply(lambda x: len(x) < 2)                               # Criar um índice booleano para filtrar os elementos com comprimento menor que 2
    s = s.drop(idx[idx].index)                                        # Remover os elementos com comprimento menor que 2
    s = s.apply(remove_stopwords)                                     # remover as stopwords: 
    idx = s.apply(lambda x: x == '')                                  # Criar um índice booleano para filtrar os elementos vazios
    s = s.drop(idx[idx].index)                                        # Remover os elementos vazios
    
    return s

def frequency_words(x):
    # Cria um dicionário vazio para armazenar os contadores
    contadores = {} 
    
    for palavra in x:
        if palavra in contadores:                   # Se a palavra já estiver no dicionário, incrementa o contador
            contadores[palavra] += 1
        else:                                       # Se a palavra não estiver no dicionário, adiciona-a e define o contador como 1
            contadores[palavra] = 1

    return contadores

In [3]:
# Abrindo o arquivo:

df = pd.read_json('C:/Users/JVoigt/OneDrive - Universität für Weiterbildung Krems/Dokumente/Python Scripts/champI4.0ns/gesetze_sammlung/eu_laws_concatenado.json')

In [4]:
# GROUPING DIFFERENT TYPES OF NORMATIVE DOCUMENTS:
# pd.unique(df.gesetzt)

timber_regulation = ['eu_land_use_change', 'eu_timber_regulation']
data_and_ki_regulation = ['Daten-Governance-Rechtsakt', 'Data Act', 'eu_digital_markets_act', 'eu_horizon','eu_regulation_ai' ]
clima = [ 'eu_gruene_deal_com','Europäisches Klimagesetz']

#timber:

df_timber = df.query("gesetzt in @timber_regulation")
df_data_ki = df.query("gesetzt in @data_and_ki_regulation")
df_clima = df.query("gesetzt in @clima")

First problem: the corpus from the Timber laws are too small for the analisys with Berttopic ```N == 56```. Let'S first continue with the other normative documents

In [5]:
### 1. Olhando a frequência das palavras para incrementar as stopwords:
# DATA AND KI #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_data_ki['text'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

# criando um dicionário compreensao:
most_common = {k: v for k, v in freq.items() if v > 100}

#most_common

In [6]:
# Prepositions and pronouns
prep = ['denen', 'fur', 'uber', 'deren', 'sowie']

# Worte, die auf die Europäische Union anspielen:
ue_wort = ['kommission' , 'mitgliedstaaten' , 'union', 'eu', 'nationalen']

# Worte, die auf Gesetzte anspielen:
gesetz_wort = ['artikel', 'absatz', 'genannten', 'verordnung' , 'artikels', 'kapitel', 'gemäß' ]

german_stop_words.extend(prep)
german_stop_words.extend(ue_wort)
german_stop_words.extend(gesetz_wort)

In [7]:
# Revisando
s = pd.Series(df_data_ki['text'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

# Acho que está ok! Agora tenho que ver isso no topic modeling mesmo.

daten 455
maßnahmen 191
stellen 182
gemaß 174
anbieter 170
torwachter 168
anforderungen 153
informationen 133
unternehmen 132
einschließlich 129
konnen 124
rahmen 124
stelle 123
zustandigen 107
personen 106
sinne 104
nutzer 104
innerhalb 103
betreffenden 103
festgelegten 100
bedingungen 100
organisationen 97
bezug 95
verfahren 92
gegebenenfalls 91
recht 90
nutzung 90
programms 90
datenvermittlungsdiensten 89
behorden 88
buchstabe 88
europaischen 87
verpflichtungen 86
behörden 86
insbesondere 85
ergebnisse 85
zugang 84
gilt 82
wurde 79
unterstutzung 77
dateninhaber 77
erlassen 76
datenaltruistischen 76
mitgliedstaat 75
zusammenhang 75
iii 75
behorde 74
maßnahme 74
begunstigten 74
erforderlich 73
datenvermittlungsdienste 71
wurden 71
plattformdienste 71
bewertung 70
beschluss 68
nationale 67
person 67
anwendung 67
zentralen 67
anhang 67
sofern 66
grundlage 66
zuständigen 66
programm 65
weiterverwendung 64
einhaltung 64
vorliegenden 62
soweit 62
einrichtung 60
zwecke 60
frist 60
dienste 6

In [8]:
# Testando com o segundo banco:

# Clima #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_clima['text'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

maßnahmen 98
rahmen 75
gemaß 64
europaischen 62
konnen 53
zusammenarbeit 51
cybersicherheit 49
einschließlich 47
insbesondere 43
investitionen 41
verwirklichung 39
unterstutzen 36
klimaneutralitat 36
unternehmen 34
mussen 33
ziel 31
beitragen 31
sollten 31
daruber 31
fordern 31
entwicklung 30
neue 30
ziele 30
strategie 29
grunen 27
einschlagigen 27
vorschlagen 27
hinaus 27
europa 26
europaische 26
verfahren 26
fortschritte 26
energie 25
einklang 25
erreichen 25
klimawandel 25
dabei 24
internationalen 24
sicherheit 23
wurde 23
internet 23
technologien 23
internationale 23
bereich 23
ubergang 23
gemeinsamen 23
nachhaltige 23
paris 23
außerdem 22
rechtsvorschriften 22
gemeinsame 22
klima 22
bedeutung 21
cyberraum 21
rolle 21
forderung 21
digitalen 20
unterstutzung 20
aufbau 20
umsetzung 20
zusammenarbeiten 20
auswirkungen 19
globalen 19
jahr 19
informationen 19
einrichtungen 19
sektoren 19
grundlage 19
bieten 19
langfristigen 19
ebene 18
schutz 18
starken 18
dafur 18
sorgen 18
arbeiten 18

In [9]:
# vou adicionar a palavra 'europaischen' aos stopwords:

german_stop_words.extend(['europaischen'])

In [10]:
# Testando com o terceiro banco:

# Timber #

# Concatenando todas as linhas e depois criando uma série com as palavras
# lembrando que AQUI eu quero só a frequencia das palavras, por isso elas podem compor a mesma série
s = pd.Series(df_timber['text'].sum().split())

# Limpando:
x = clean_corpus(s)

# Vendo frequencia:
freq = frequency_words(x)

for key, value in sorted(freq.items(), key=lambda item: item[1], reverse=True):
    print(key, value)

zeitraum 42
holz 40
mitgliedstaat 35
emissionen 31
folgende 28
anhang 27
vorliegenden 23
gemaß 23
fassung 20
einschließlich 18
flächen 18
erhält 17
waldflächen 16
zustandigen 16
sonstige 15
rechtsakte 15
holzerzeugnisse 15
holzerzeugnissen 15
behorden 15
maßnahmen 14
marktteilnehmer 14
kontrollen 14
buchstabe 13
grünland 13
wurde 13
flexibilitätsregelungen 13
insbesondere 13
sorgfaltspflichtregelung 13
landnutzung 12
treibhausgasen 12
zielvorgaben 12
gegebenenfalls 12
anwendung 12
informationen 12
geändert 11
abbau 11
folgenden 11
bewirtschaftete 11
artikeln 11
delegierten 11
uberwachungsorganisation 11
konnen 11
abl 11
abbaus 10
gilt 10
feuchtgebiet 10
umgewandelt 10
gesamtemissionen 10
jährlichen 10
bewertung 10
ausgleich 10
rat 10
eg 10
verfahren 10
erlassen 9
wälder 9
parlament 9
verkehr 9
folgt 8
verpflichtungen 8
einhaltung 8
nettoabbau 8
rates 8
siedlung 8
fläche 8
dezember 8
buchstaben 8
übersteigen 8
tonnen 8
coäquivalent 8
jahre 8
jahr 8
grundlage 8
sofern 8
betreffenden 8
de

In [18]:
with open("german_stopwords.json", "w") as f:
    json.dump(german_stop_words, f)

# # Carregue a lista a partir do arquivo
# with open("lista.json", "r") as f:
#     lista_carregada = json.load(f)  

In [24]:
# Salvando os dfs

dfs = [df_clima, df_data_ki, df_timber]

for d in dfs:
    d.reset_index()

df_clima.to_csv('df_clima.csv')
df_data_ki.to_csv('df_data_ki.csv')
df_timber.to_csv('df_timber.csv')